In [ ]:
!pip install evaluate==0.4.3
!pip install llama-cpp-python==0.1.9
!pip install pinecone-client==5.0.1
!pip install langchain_community==0.2.16
!pip install langchain-chroma==0.1.4
!pip install chromadb==0.5.11
!pip install sentence-transformers==3.1.1

In [ ]:
from langchain_community.document_loaders import PDFMinerLoader, TextLoader, CSVLoader, UnstructuredWordDocumentLoader, UnstructuredHTMLLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from multiprocessing.pool import ThreadPool
from langchain_chroma.vectorstores import Chroma
from langchain.schema import Document
from chromadb.config import Settings
from llama_cpp import Llama
from evaluate import load
from typing import Any
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
import numpy as np
import statistics
import pinecone
import glob
import os

In [ ]:
from sentence_transformers import SentenceTransformer

#Лабораторная работа №5

##Declaring constant

In [ ]:
# Словарь, сопоставляющий расширения файлов с соответствующими загрузчиками данных и их параметрами
LOADER_MAPPING = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".pdf": (PDFMinerLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"}),
}

In [ ]:
# Параметры конфигурации для векторного поиска и разделения текста
INDEX_NAME = "VDB"  # Название индекса для хранения векторных представлений
EMBEDDINGS = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'  # Название модели эмбеддингов, используемой для векторизации текстов
SIZE = 250  # Размер фрагмента текста для разделения документов
OVERLAP = 50  # Перекрытие между фрагментами текста для обеспечения контекста

##Loader

In [ ]:
import itertools
import re

pattern = r"{price_pattern}|{abbr_patterns}|({phone_pattern})|({email_pattern})|(\'?[\w\-]+)|([^A-Za-z0-9 \n])"
sent_pattern = r"((?<=\.|\?|!|\;))({abbr_patterns})\s"

phone_pattern = r"\+?[0-9] ?\(?[0-9]+\)?[0-9 -]+"
# [\+]?[(]?[0-9]{3}[)]?[-\s\.]?[0-9]{3}[-\s\.]?[0-9]{4,6}
email_pattern = r"[^@ \t\r\n]+@[^@ \t\r\n]+\.[^@ \t\r\n]+"
price_pattern = r"(\$ ?\d*\.?\d+)|(\d*\.?\d+ ?\$)"

english_abbr = ["Mr.", "Mrs.", "Mss.", "Ms.", "Dr."]
english_abbr = [x.replace(".", "\.") for x in english_abbr]
english_abbr.extend(map(lambda x: x.lower(), english_abbr.copy()))

sent_pattern = sent_pattern.format(abbr_patterns="".join(map(lambda x: fr"(?<!{x})", english_abbr)))
sent_pattern = re.compile(sent_pattern)

def split_to_sentence(text: str) -> list[str]:
    return list(filter(lambda x: len(x) if x else False, sent_pattern.split(text)))

In [ ]:
def load_dataset(split_type="train", n: int | None = None, dataset_path = "../../assets/{split_type}.csv", random_state=42) -> pd.DataFrame:
    assert split_type == "train" or split_type == "test"
    dataset_path = dataset_path.format(split_type=split_type)
    if not os.path.exists(dataset_path):
        splits = {'train': 'yelp_review_full/train-00000-of-00001.parquet',
                  'test': 'yelp_review_full/test-00000-of-00001.parquet'}
        df = pd.read_parquet("hf://datasets/Yelp/yelp_review_full/" + splits[split_type])
        df.to_csv(dataset_path, index=False)
    else:
        df = pd.read_csv(dataset_path)
    if n is None:
        return df
    else:
        return train_test_split(df, train_size=n, stratify=df["label"], random_state=random_state)[0]

In [ ]:
def process_df(df: pd.DataFrame) -> list[tuple[list[str], str|int]]:
    data = []
    meta = []
    ids = []
    for idx, row in df.iterrows():
        label, text = row["label"], row["text"]
        chunks = splitter.split_document(text)
        data.extend(chunks)
        meta.extend([{"label": label} for _ in range(len(chunks))])
        ids.extend([f"{idx}_{i}" for i in range(len(chunks))])
    return data, meta, ids

def dataset_batch_iter(df, batch_size):
    for df_b in np.array_split(df, batch_size):
        yield process_df(df_b)
    return

In [ ]:
# Класс для загрузки документов из различных источников, поддерживающий работу с разными форматами файлов
class Loader:
    def load_single_document(self, file_path: str):
        return

    def load_documents(self, source_dir: str):
        pass  # Метод для загрузки всех документов из указанной директории

##Splitter

In [ ]:
# Класс для разделения документов на фрагменты определённого размера с заданным перекрытием
class Splitter:
    def __init__(self, chunk_size, chunk_overlap):
        assert chunk_size > chunk_overlap
        self.chunk_size = chunk_size
        self.chunk_overlap=chunk_overlap

    def split_document(self, document: str):
        # Метод для разделения переданных документов на фрагменты
        doc_sents = []
        for sent in split_to_sentence(document):
          for i in range(0, len(sent), self.chunk_size-self.chunk_overlap):
            start, end = i, i+self.chunk_size
            doc_sents.append(sent[start: end])
        return doc_sents

In [ ]:
splitter=Splitter(SIZE, OVERLAP)

In [ ]:
# splitter.split_document("I love driving and I dont loke kaksd asdkf")

In [ ]:
df = load_dataset(split_type="train", n=10, dataset_path='./assets/{split_type}.csv')
for i in dataset_batch_iter(df, batch_size = 2):
  print(i)
  break

##Vector database

In [ ]:
# Базовый класс для работы с коллекцией документов, поддерживающий добавление, поиск и очистку данных
class Collector:
    def add(self, texts: list[str], metadatas: list[dict]):
        pass  # Метод для добавления текстов и связанных с ними метаданных в коллекцию

    def add_from_directory(self, dir_path: str):
        pass  # Метод для добавления документов в коллекцию из указанной директории

    def get(self, search_strings: list[str], n_results: int) -> list[Document]:
        pass  # Метод для поиска документов по строкам запроса с ограничением на количество результатов

    def get_documents(self, search_string: str, n_results: int, score_threshold: float) -> list[Document]:
        pass  # Метод для поиска документов с учётом порога релевантности и количества возвращаемых результатов

    def clear(self):
        pass  # Метод для очистки коллекции документов

In [ ]:
# Базовый класс для создания эмбеддингов, обеспечивающий интерфейс для получения модели эмбеддингов
class Embedder:
    def __init__(self, model_name):
        pass  # Инициализация эмбеддера

    def get_embedding(self, sent):
        pass  # Метод для получения модели эмбеддингов, которая будет использоваться для векторизации текстов

In [ ]:
class SentenceEmbedder(Embedder):
    def __init__(self, model_name: str = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"):
       self.model = SentenceTransformer(model_name)
    def get_embedding(self, sent):
        # Метод для получения модели эмбеддингов, которая будет использоваться для векторизации текстов
        return self.model.encode(sent).tolist()
    def __call__(self, input):
        return self.get_embedding(input)

In [ ]:
class ChromaCollector(Collector):
    def __init__(self, name_prefix, root_path, embeddnig_fn, distance_fn):
      self.client = chromadb.PersistentClient(path=root_path)
      self.distance_fn = distance_fn
      self.embedding_fn = embeddnig_fn
      self._collection_name = name_prefix + self.distance_fn
      self.database = self.get_database()

    def get_database(self):
      return self.client.get_or_create_collection(
            self._collection_name,
            metadata={"hnsw:space": self.distance_fn},
            embedding_function=self.embedding_fn
        )

    def load_dataset(self, df: pd.DataFrame, batch_size=128) -> None:
        for chunks, metas, ids in tqdm(dataset_batch_iter(df, batch_size = batch_size), total=math.ceil(df.shape[0] / batch_size), desc="loading to the DB"):
          self.database.add(
                documents=chunks,
                metadatas=metas,
                ids=ids
            )

    def query(self, query, n_results: int, query_texts=None, where=None, where_document=None):
        return self.database.query(
            n_results=n_results,
            query_texts=query_texts,
            query_embeddings=self.embedding_fn(query),
            where=where,
            where_document=where_document
        )

    def clear(self):
        self.client.delete_collection(self._collection_name)

###Implementation vector database

In [ ]:
path_to_index = '/VDB' #@param {type:"string"}
path_to_df = './assets/{split_type}.csv' #@param {type:"string"}

In [ ]:
import chromadb
import math

In [ ]:
df = load_dataset(split_type="train", n=10_000, dataset_path=path_to_df)
emedder = SentenceEmbedder()

In [ ]:
database_cos = ChromaCollector("my_db", path_to_index, emedder, "cosine")

In [ ]:
database_cos.load_dataset(df)

##Search

In [ ]:
query = 'What is your favorite food?' #@param {type:"string"}
n_results = 5 #@param {type:"integer"}
# score_threshold = 0.5 # @param {type:"slider", min:0, max:1, step:0.1}

result = database_cos.query(query, n_results=5)
for dist, document, meta in zip(result["distances"][0], result["documents"][0], result["metadatas"][0]):
  print(f"{dist:0.2f}  {meta['label']}  {document}")

In [ ]:
query = 'What is your favorite restaurant?' #@param {type:"string"}
n_results = 5 #@param {type:"integer"}

result = database_cos.query(query, n_results=5)
for dist, document, meta in zip(result["distances"][0], result["documents"][0], result["metadatas"][0]):
  print(f"{dist:0.2f}  {meta['label']}  {document}")

In [ ]:
query = 'What do you think about this doctor?' #@param {type:"string"}
n_results = 5 #@param {type:"integer"}

result = database_cos.query(query, n_results=5)
for dist, document, meta in zip(result["distances"][0], result["documents"][0], result["metadatas"][0]):
  print(f"{dist:0.2f}  {meta['label']}  {document}")

##Evaluation

In [ ]:
# Класс для оценки работы коллектора, предоставляющий функционал для поиска, оценки и расчета статистики по результатам
class CollectorEvaluator:
    def __init__(self, collector: Collector, n_top=100):
        pass  # Инициализация коллектора и параметра n_top для ограничения числа возвращаемых результатов

    def explore_collector(self, text):
        pass  # Метод для поиска документов в коллекторе на основе текста запроса

    def eval(self, query, answer):
        pass  # Метод для оценки корректности найденных документов на основе запроса и правильного ответа

    def calculate_statistics(self, data):
        pass  # Метод для расчета статистических показателей (например, минимальное, максимальное, среднее значение)

    def explore_and_calculate(self, data):
        pass  # Метод для проведения поиска по данным и расчета статистики на основе результатов

In [ ]:
n_docs = 100 #@param {type:"integer"}
n_top = 100 #@param {type:"integer"}

In [ ]:
df_qa = df.iloc[:n_docs,]

In [ ]:
def find_id_index(result_ids, my_id, trunc=False):
  for idx, rid in enumerate(result_ids):
    if trunc:
      if rid.split("_")[0]==my_id.split("_")[0]:
        return idx
    else:
      if rid == my_id:
        return idx
  return

In [ ]:
find_id_index(["111", "123"], "123", trunc=True)

In [ ]:
find_id_indexes = []
for chunks, metas, ids in tqdm(dataset_batch_iter(df_qa, batch_size = 1), desc="qa DB"):
  for chunk, meta, id_ in zip(chunks, metas, ids):
    results = database_cos.query(query=chunk, n_results=n_top)
    ch_id = find_id_index(results["ids"][0], id_, trunc=False)
    find_id_indexes.append(ch_id)

In [ ]:
indexes = np.array(find_id_indexes, dtype="float64")

print("Средняя позиция документа:" ,np.mean(indexes[~np.isnan(indexes)]))
print("Количество не найденных документов:", np.count_nonzero(np.isnan(indexes)))

In [ ]:
queries = [ # text_query, doc_id
    ("What color is a parking pass?", 121392),
    ("Does the discount compensate the workers incompetent?", 340905),
    ("How long I was a Centurylink customer?",561808),
    ("What was a spicy level for the Panang Curry", 153961),
    ("How many times does the servers came back?", 596493),
    ("Where is a good Ted Wien's store located?", 257236),
    ("Which DJ played on Paul Oakenfield night?", 213914),
    ("How many stars does the Carvel had last year?", 418978),
    ("What item did a man from Arizona buy two at a time?", 33736),
    ("What dish\drink was cancelled because of its taste?", 335445)
]

In [ ]:
find_id_indexes = []
for data in tqdm(queries):
  query, id_ = data
  results = database_cos.query(query=query, n_results=n_top)
  ch_id = find_id_index(results["ids"][0], str(id_), trunc=True)
  find_id_indexes.append(ch_id)

In [ ]:
indexes = np.array(find_id_indexes, dtype="float64")

print("Средняя позиция документа:" ,np.mean(indexes[~np.isnan(indexes)]))
print("Количество не найденных документов:", np.count_nonzero(np.isnan(indexes)))